In [1]:
import pandas as pd
import numpy as np

In [2]:
## read df
df_1st_hour = pd.read_csv(r'C:\Users\arvinmvn\Desktop\Split Throughput\recent\1st hr.csv')
df_tm = pd.read_excel(r'C:\Users\arvinmvn\Desktop\Split Throughput\Radhika Org TM-OM-SM mapping.xlsx')
df_1st_hour.head()

,DA Login,First Login Time,Current Status,Skill Value ID,UPH,AHT (In sec),Velocity,Utilization %,NPT %,Processed Units,...,Manual Skip Hours,Non Mercury Processing Hours,Staff Hours,NPT,Idle No Data,Training Hours,Lost Hours,System Issue,System Latency,Schedule Non Adherence
0,aanbub,9/24/2021 1:28,mercury_production,ads_central_skill_type_PresenceDetectionSpecia...,9.375,26.280000,136.986301,5.9783,0.0,75,...,0.0,-0.0017,8,0.0,0.0,0,0.0000,0,0.0054,0.0000
1,aarjk,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,0,...,0.0,0.0000,8,0.0,0.0,0,0.0000,0,0.0000,0.0000
2,abdarsha,NaN,project,ads_central_skill_type_core_transcription,0.000,NaN,NaN,NaN,NaN,0,...,0.0,0.0000,8,0.0,0.0,0,0.0000,0,0.0000,0.0000
3,adithia,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,0,...,0.0,0.0000,8,0.0,0.0,0,0.0000,0,0.0000,0.0000
4,adityna,9/24/2021 1:29,mercury_production,NaN,2.125,53.131765,67.756078,3.0442,0.0,17,...,0.0,0.0039,8,0.0,0.0,0,0.0798,0,0.0000,0.0798


In [3]:
## insert time range column
df_1st_hour.insert(2, "Time Range",'07-08')

In [4]:
## insert shift
df_1st_hour.insert(3, "Shift",'Morning')

In [5]:
## insert process units
df_1st_hour['Processed Units']=df_1st_hour['Processed Units']

In [6]:
## insert manual skip hours
df_1st_hour['Manual Skip Hours']=df_1st_hour['Manual Skip Hours']

In [7]:
## insert AHT

df_1st_hour['AHT']=3600*((df_1st_hour['Manual Skip Hours']+df_1st_hour['Processed Hours'])/df_1st_hour['Processed Units'])

In [8]:
## combine DFs
all_df_list = [df_1st_hour]
##join all sheets
df = pd.concat(all_df_list)

In [9]:
## Merge DF
df_merge=pd.merge(df,df_tm,on="DA Login",how="left")
df_merge['Time Range'].unique()

array(['07-08'], dtype=object)

In [10]:
## Pick out necessary values

df_merge=df_merge[['DA Login','First Login Time','Processed Units','Processed Hours','Manual Skip Hours','AHT','Time Range','Shift','TM','OM']]
df_merge.head()

,DA Login,First Login Time,Processed Units,Processed Hours,Manual Skip Hours,AHT,Time Range,Shift,TM,OM
0,aanbub,9/24/2021 1:28,75,0.5475,0.0,26.280000,07-08,Morning,sekal,jagarekh
1,aarjk,NaN,0,0.0000,0.0,NaN,07-08,Morning,kkathire,mryjos
2,abdarsha,NaN,0,0.0000,0.0,NaN,07-08,Morning,czvij,rbbashee
3,adithia,NaN,0,0.0000,0.0,NaN,07-08,Morning,shawnpet,mryjos
4,adityna,9/24/2021 1:29,17,0.2509,0.0,53.131765,07-08,Morning,kkathire,mryjos


In [11]:
## Create Processed Units table

table_pu = pd.pivot_table(df_merge, values=['Processed Units'], index=['DA Login'],columns=['Time Range'],margins=True,
               margins_name='Total', aggfunc={'Processed Units':np.sum,} , fill_value=0)
table_pu.head()

Processed Units      
Time Range           07-08 Total
DA Login                        
aanbub                  75    75
aarjk                    0     0
abdarsha                 0     0
adithia                  0     0
adityna                 17    17

In [12]:
## Create Manual Skips Hours table

table_ms = pd.pivot_table(df_merge, values=['Manual Skip Hours'], index=['DA Login'],columns=['Time Range'],margins=True,
               margins_name='Total', aggfunc={'Manual Skip Hours':np.sum,} , fill_value=0)
table_ms.head()

Manual Skip Hours      
Time Range             07-08 Total
DA Login                          
aanbub                   0.0   0.0
aarjk                    0.0   0.0
abdarsha                 0.0   0.0
adithia                  0.0   0.0
adityna                  0.0   0.0

In [13]:
## Create AHT table

table_aht = pd.pivot_table(df_merge, values=['AHT'], index=['DA Login'],columns=['Time Range'],margins=True,
               margins_name='Total', aggfunc={'AHT':np.mean,} , fill_value=0)
table_aht.head()

AHT           
Time Range      07-08      Total
DA Login                        
aanbub      26.280000  26.280000
adityna     53.131765  53.131765
aharinat     1.447552   1.447552
akshavig    45.142326  45.142326
amerud      35.085957  35.085957

In [14]:
## Create Processed Units table

table = pd.pivot_table(df_merge, values=['Processed Units','Processed Hours','Manual Skip Hours','AHT'], index=['DA Login','Time Range'],margins=True,
               margins_name='Total', aggfunc={'Processed Units':np.sum,
                                                        'Processed Hours':np.sum,
                                                        'Manual Skip Hours':np.sum,
                                                        'AHT':np.mean,} , fill_value=0)
table.head()

,,AHT,Manual Skip Hours,Processed Hours,Processed Units
DA Login,Time Range,,,,
aanbub,07-08,26.280000,0.0,0.5475,75
aarjk,07-08,0.000000,0.0,0.0000,0
abdarsha,07-08,0.000000,0.0,0.0000,0
adithia,07-08,0.000000,0.0,0.0000,0
adityna,07-08,53.131765,0.0,0.2509,17


In [15]:
## create list for email iteration

list=df_merge['DA Login'].unique()
list=list[0:3]
len(list)

3

In [16]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

    
for i in range (0,len(list)):
    
    
    
    table_pu = pd.pivot_table(df_merge, values=['Processed Units'], index=['DA Login'],columns=['Time Range'],margins=True,
               margins_name='Total', aggfunc={'Processed Units':np.sum,} , fill_value=0)
    
    table_ms = pd.pivot_table(df_merge, values=['Manual Skip Hours'], index=['DA Login'],columns=['Time Range'],margins=True,
               margins_name='Total', aggfunc={'Manual Skip Hours':np.sum,} , fill_value=0)
    
    table_aht = pd.pivot_table(df_merge, values=['AHT'], index=['DA Login'],columns=['Time Range'],margins=True,
               margins_name='Total', aggfunc={'AHT':np.mean,} , fill_value=0) 

    pd.set_option('display.max_rows', df_merge.shape[0]+1)

    mail = outlook.CreateItem(0)

    mail.Subject = 'ADS Hourly Throughput Summary Report'
    mail.To= 'arvinmvn@amazon.com'
    #mail.To = list[i]+'@amazon.com' 
    #mail.CC = table_process_count.index.get_level_values('Level 1 Manager')[j] + '@amazon.com'
        
    
## i-> position of data in dataframe
    mail.HTMLBody =f"""<h3>{'Hi ' + list[i]}</h3> """ + f"""<h3>PFB Hourly summary upto to {df_1st_hour['Time Range'].unique()} </h3>""" + '''<h3>Processed Units Table:-</h3>
                   {}'''.format(table_pu[i:i+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render())+ '''<h3>Manual Skip Hours Table:-</h3>
                   {}'''.format(table_ms[i:i+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3>AHT Table:-</h3>
                   {}'''.format(table_aht[i:i+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3> Thanks and Regards </h3>''' + '''<h3> Arvinth M.V.N </h3>'''
  
  
    mail.Send()

In [17]:
## Store output for next iteration

df_merge.to_excel(r'C:\Users\arvinmvn\Desktop\Hourly Output Folder\07_08.xlsx',index=False)